# 220. CNN Basic

## Convolution

<img src='conv_layer.gif' height=60% width=60% />

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

<img src = "convolution.JPG" width = 500, align = "center">

In [2]:
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, stride=1, padding=0, bias=False)
conv

Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1), bias=False)

- <code>nn.Conv2d</code> 의 parameter 는 random 하게 초기화되고, 훈련 과정에서 학습됨.  

In [3]:
conv.state_dict()

OrderedDict([('weight',
              tensor([[[[ 0.3640, -0.2951],
                        [-0.0612,  0.1379]]]]))])

- 간단한 계산 예시를 위해 kernel parameter 값을 임의로 수정

In [4]:
conv.state_dict()['weight'][0][0] = \
                        torch.tensor([[1., 2.],
                                      [3., 4.]])
conv.state_dict()

OrderedDict([('weight',
              tensor([[[[1., 2.],
                        [3., 4.]]]]))])

- sample image를 4x4 크기로 생성  

- ``torch.nn`` 은 미니 배치(mini-batch)만 지원하므로, `nnConv2D` 는 `nSamples x nChannels x Height x Width` 의
    4차원 Tensor를 입력으로 한다. 하나의 샘플만 있다면, `input.unsqueeze(0)` 을 사용해서 가짜 차원을 추가한다.

In [5]:
sample_image = torch.tensor([
                             [1, 0, 1, 0],
                             [0, 1, 1, 0],
                             [1, 0, 1, 0],
                             [1, 0, 1, 1]], dtype=torch.float)

sample_image = sample_image.unsqueeze(0).unsqueeze(0)
print(sample_image.numpy())

[[[[1. 0. 1. 0.]
   [0. 1. 1. 0.]
   [1. 0. 1. 0.]
   [1. 0. 1. 1.]]]]


- convolution 결과

In [6]:
z = conv(sample_image)

print(z.shape)
print()
print(z.detach().numpy())

torch.Size([1, 1, 3, 3])

[[[[5. 9. 4.]
   [5. 7. 4.]
   [4. 6. 8.]]]]


### Convolutional Layer 의 output size 계산

<img src = "padding.JPG" width = 500, align = "center">

$$ \text{output width} = \lfloor\frac{W - F_w + 2P}{S_w} + 1\rfloor  $$
$$ \text{output height} = \lfloor\frac{H - F_h + 2P}{S_h} + 1 \rfloor$$
$W - \text{input width}$  
$F_{w,h} - \text{filter width/height}$  
$S_{w,h} - \text{stride width/height}$  
$P - padding$

`conv output size = (input size - filter size + 2 * padding size) / stride size + 1`

In [7]:
def output_size(W, F, P, S, poolsize=1):
    size = (W - F + 2*P)/S + 1
    return size if poolsize == 1 else size / poolsize

In [8]:
image1 = torch.tensor([
                       [0, 1, 2],
                       [3, 4, 5],
                       [6, 7, 8]], dtype=torch.float)

image1 = image1.unsqueeze(0).unsqueeze(0)
print(image1.size())
print(image1.numpy())

torch.Size([1, 1, 3, 3])
[[[[0. 1. 2.]
   [3. 4. 5.]
   [6. 7. 8.]]]]


- no padding 의 경우 output image

In [9]:
F = 2
S = 1
P = 0

conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=F, stride=S, bias=False)

conv1.state_dict()['weight'][0][0] = torch.tensor([[0., 1.], [2., 3.]])

z1 = conv1(image1)

print("입력 image size =", image1.size())
print("convolution 후 image size =", z1.size())
print("Convolution 출력 data =", z1.detach().numpy())
print()

H = image1.size()[2]
W = image1.size()[3]
print("출력 Hight = ", (H - F + 2*P) // S + 1)
print("출력 Width = ", (W - F + 2*P) // S + 1)

입력 image size = torch.Size([1, 1, 3, 3])
convolution 후 image size = torch.Size([1, 1, 2, 2])
Convolution 출력 data = [[[[19. 25.]
   [37. 43.]]]]

출력 Hight =  2
출력 Width =  2


In [10]:
output_size(W, F, P, S, poolsize=1)

2.0

- padding=1 인 경우의 output image

In [11]:
F = 2
S = 1
P = 1

conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=F, stride=S, padding=P, bias=False)

conv2.state_dict()['weight'][0][0] = torch.tensor([[0., 1.], [2., 3.]])

z2 = conv2(image1)

print("입력 image size =", image1.size())
print("convolution 후 image size =", z2.size())
print("Convolution 출력 data =", z2.detach().numpy())
print()

print("출력 Hight = ", (H - F + 2*P) // S + 1)
print("출력 Width = ", (W - F + 2*P) // S + 1)

입력 image size = torch.Size([1, 1, 3, 3])
convolution 후 image size = torch.Size([1, 1, 4, 4])
Convolution 출력 data = [[[[ 0.  3.  8.  4.]
   [ 9. 19. 25. 10.]
   [21. 37. 43. 16.]
   [ 6.  7.  8.  0.]]]]

출력 Hight =  4
출력 Width =  4


In [12]:
output_size(W, F, P, S, poolsize=1)

4.0

## Pooling

- Max Pooling - <code>torch.nn.MaxPool2d</code>  
- Average Pooling - <code>torch.nn.AvgPool2d</code>

In [18]:
# 4x4 크기의 2D 텐서를 생성합니다. 이 텐서는 예를 들어 하나의 그레이스케일 이미지를 나타낼 수 있습니다.
image2 = torch.tensor([
                       [1, 1, 1, 4],
                       [2, 6, 5, 8],
                       [3, 2, 1, 0],
                       [1, 1, 3, 5]], dtype=torch.float)

# image2 텐서의 형태를 출력합니다. 결과는 [4, 4]로, 4x4 크기의 2D 텐서임을 나타냅니다.
print(image2.shape)

# unsqueeze(0)을 사용하여 첫 번째 차원(배치 차원)을 추가합니다.
# 두 번째 unsqueeze(0) 호출로 채널 차원을 추가합니다.
# 이렇게 함으로써 2D 이미지 텐서를 [배치 크기, 채널 수, 높이, 너비] 형태의 4D 텐서로 변환합니다.
image2 = image2.unsqueeze(0).unsqueeze(0)

# 변경된 image2 텐서의 형태를 출력합니다. 결과는 [1, 1, 4, 4]로,
# 1개의 배치에 속한 1개의 채널을 가진 4x4 크기의 이미지임을 나타냅니다.
print(image2.shape)

# 변환된 image2 텐서를 출력합니다.
image2

torch.Size([4, 4])
torch.Size([1, 1, 4, 4])


tensor([[[[1., 1., 1., 4.],
          [2., 6., 5., 8.],
          [3., 2., 1., 0.],
          [1., 1., 3., 5.]]]])

In [22]:
# 2x2 크기의 커널을 사용하고, stride를 2로 설정한 MaxPooling을 정의합니다.
max1 = torch.nn.MaxPool2d(2, stride=2)

# MaxPooling을 이미지에 적용하고 2x2 영역에서 최대값을 선택하여 이미지의 크기를 줄입니다.
print(max1(image2))

tensor([[[[6., 8.],
          [3., 5.]]]])


In [23]:
# 2x2 크기의 커널을 사용하고, stride를 2로 설정한 평균 풀링(Average Pooling)을 정의합니다.
max2 = torch.nn.AvgPool2d(2, stride=2)

# 평균 풀링을 이미지에 적용하고 2x2 영역의 평균값을 계산하여 이미지의 크기를 줄입니다.
print(max2(image2))

tensor([[[[2.5000, 4.5000],
          [1.7500, 2.2500]]]])


### Multiple Channels

In [24]:
# 입력 채널이 1개, 출력 채널이 3개인 합성곱 레이어
# kernel_size는 커널(또는 필터)의 크기를 2x2로 설정합니다.
# bias=False는 합성곱 연산 시 편향(bias)을 추가하지 않겠다는 의미입니다.
conv = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=2, bias=False)

# 이 예제에서는 출력 채널이 3개이므로, 3개의 다른 커널이 각각 적용되어 3개의 특징 맵을 출력
result = conv(sample_image)
print(result)

tensor([[[[ 0.6541,  0.4197,  0.5572],
          [ 0.2552,  0.8417,  0.5572],
          [ 0.5572,  0.3520,  0.7217]],

         [[ 0.0839, -0.3128,  0.2103],
          [-0.2599, -0.2494,  0.2103],
          [ 0.2103, -0.3863,  0.1574]],

         [[-0.4985, -1.2642, -0.5449],
          [-0.8941, -0.9760, -0.5449],
          [-0.5449, -0.8476, -0.9151]]]], grad_fn=<ConvolutionBackward0>)


### Flatten

- ``torch.flatten(x, 1)``

In [25]:
t = torch.tensor([
                   [[1, 2],
                   [3, 4]],
                  [[5, 6],
                   [7, 8]]])
print(t.shape)
print(t)

torch.Size([2, 2, 2])
tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [27]:
# torch.flatten 함수는 텐서를 평탄화합니다. 여기서 시작 차원(start_dim)을 1로 설정합니다.
# 이는 첫 번째 차원(배치 크기)을 유지한 채 나머지 차원을 평탄화하여 2차원 텐서를 생성합니다.
# 예를 들어, [배치 크기, 채널 수, 높이, 너비] 형태의 텐서는 [배치 크기, 채널 수*높이*너비]로 평탄화됩니다.
flattened_t = torch.flatten(t, 1)

print(flattened_t.shape)
print(flattened_t)

torch.Size([2, 4])
tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])


In [29]:
# flatten() 과 동일 result
# t를 [배치 크기, 나머지 모든 요소의 수]의 2차원 텐서로 변환합니다.
# 예를 들어, [배치 크기, 채널 수, 높이, 너비] 형태의 텐서는 [배치 크기, 채널 수*높이*너비]로 변환됩니다.
reshaped_t = t.view(t.size()[0], -1)
print(reshaped_t.shape)
print(reshaped_t)

torch.Size([2, 4])
tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])
